# Analyzing borrowers’ risk of defaulting

This project is prepares a report for a bank’s loan division. The goal is to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness. My report is used for building the credit score of a potential customer. 

## Open the data file and inspect general information: 

In [1]:
# Loading all the libraries
import pandas as pd
import numpy as np
import scipy.stats
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the data
df = pd.read_csv('/datasets/credit_scoring_eng.csv')

In [3]:

# Check for rows and columns of the dataset
df.shape

(21525, 12)

## Task 1. Data exploration

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan

In [4]:
# Print the first N rows
print(df.head(10))

   children  days_employed  dob_years            education  education_id  \
0         1   -8437.673028         42    bachelor's degree             0   
1         1   -4024.803754         36  secondary education             1   
2         0   -5623.422610         33  Secondary Education             1   
3         3   -4124.747207         32  secondary education             1   
4         0  340266.072047         53  secondary education             1   
5         0    -926.185831         27    bachelor's degree             0   
6         0   -2879.202052         43    bachelor's degree             0   
7         0    -152.779569         50  SECONDARY EDUCATION             1   
8         2   -6929.865299         35    BACHELOR'S DEGREE             0   
9         0   -2188.756445         41  secondary education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employee     0     40620.102   
1

#### There are issues with the data sample:

1. days employed are showing negative values - which is illogical.
2. spellings are not standardly formatting - for example, within the education column, capitalization and spellings are varied.
3. within the purpose column, there's a great variety of information, which might make sorting and categorization difficult.
4. unclear exactly what the 0 and 1 siginifies within "family_status_id" and "education_id". One's best guess is a boolean value indicating yes/no or true/false for these columns.

In [5]:
# Get info on data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


####  There are missing values across a few columns:

Specifically, there are missing values for "days employed" and "total income" columns. Total row number should be 21525, but these columns only show 19351 rows/values.

In [6]:
# Examine the filtered table with missing values in the the first column with missing data
missing = df.loc[:, ['days_employed','total_income']]
print(df.loc[ (df['days_employed'] == '') & (df['total_income'] == '') ] )

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


#### In the "days_employed" and "total_income" columns, when I print just the first 50 rows, I can see a few things:

1. the missing values seem symmetric, since there are corresponding NaN across these two columns.
2. all missing values are "NaN", which indicates it is not a number and is a float type, and that I can perform mathematical operations on it if needed. 

In [7]:
# Apply multiple conditions for filtering data and look at the number of rows in the filtered table.
missed = df[(df.days_employed != 0) & (df.total_income != 0)]
missed.info()
missing.dropna()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


,days_employed,total_income
0,-8437.673028,40620.102
1,-4024.803754,17932.802
2,-5623.422610,23341.752
3,-4124.747207,42820.568
4,340266.072047,25378.572
...,...,...
21520,-4529.316663,35966.698
21521,343937.404131,24959.969
21522,-2113.346888,14347.610
21523,-3112.481705,39054.888


**Intermediate conclusion**

The number of rows in the filtered table now match the number of missing values earlier, which we identified with df.info(). The size of the table has reduced down to 19351 rows, without NaN missing values.

As calculated below, the missing values account for roughly 10% of the data. It is not astronomically large, but I think it's nevertheless worthy to consider whether the missing data could be due to the specific client characteristic. Here's what I think might be the reason: 
1) the missing values for income and days_employed is 0 for SOME 'retiree'.
2) one's education level seems to have an influence on your income and days_employed: not all "secondary education" folks have a missing value in these columns, but all the missing columns store data of a person whose education is that of "secondary education"

I printed the head and tail of the dataframe, and sampled roughly 100 data points to discern for any specific client characteristic I think might be the reason. I realized it makes sense that retirees could have income and employment days equal to zero. One's education level seems to have an influence on your income and days_employed: not all "secondary education" folks have a missing value in these columns, but all the missing columns store data of a person whose education is that of "secondary education"

In [8]:
(21525-19351)/21525*100

10.099883855981417

In [9]:
# Investigate clients who do not have data on identified characteristic and the column with the missing values
print(df.head(50))
print(df.tail(50))

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42    bachelor's degree             0   
1          1   -4024.803754         36  secondary education             1   
2          0   -5623.422610         33  Secondary Education             1   
3          3   -4124.747207         32  secondary education             1   
4          0  340266.072047         53  secondary education             1   
5          0    -926.185831         27    bachelor's degree             0   
6          0   -2879.202052         43    bachelor's degree             0   
7          0    -152.779569         50  SECONDARY EDUCATION             1   
8          2   -6929.865299         35    BACHELOR'S DEGREE             0   
9          0   -2188.756445         41  secondary education             1   
10         2   -4171.483647         36    bachelor's degree             0   
11         0    -792.701887         40  secondary education             1   

In [10]:
# Checking distribution
retiree = df[df.income_type.str.startswith('r')]
retiree.info()
retiree.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3856 entries, 4 to 21521
Data columns (total 12 columns):
children            3856 non-null int64
days_employed       3443 non-null float64
dob_years           3856 non-null int64
education           3856 non-null object
education_id        3856 non-null int64
family_status       3856 non-null object
family_status_id    3856 non-null int64
gender              3856 non-null object
income_type         3856 non-null object
debt                3856 non-null int64
total_income        3443 non-null float64
purpose             3856 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 391.6+ KB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,3856.000000,3443.000000,3856.000000,3856.000000,3856.000000,3856.000000,3443.000000
mean,0.132002,365003.491245,59.063019,0.913900,0.985737,0.056017,21940.394503
std,1.014271,21069.606065,7.618526,0.510185,1.315202,0.229984,12839.512522
min,-1.000000,328728.720605,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,346649.346146,56.000000,1.000000,0.000000,0.000000,13261.031000
50%,0.000000,365213.306266,60.000000,1.000000,0.000000,0.000000,18962.318000
75%,0.000000,383231.396871,64.000000,1.000000,2.000000,0.000000,27152.069000
max,20.000000,401755.400475,74.000000,4.000000,4.000000,1.000000,117616.523000


In [11]:
retiree_dropped = retiree.dropna()
retiree_dropped.info()
retiree_dropped.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3443 entries, 4 to 21521
Data columns (total 12 columns):
children            3443 non-null int64
days_employed       3443 non-null float64
dob_years           3443 non-null int64
education           3443 non-null object
education_id        3443 non-null int64
family_status       3443 non-null object
family_status_id    3443 non-null int64
gender              3443 non-null object
income_type         3443 non-null object
debt                3443 non-null int64
total_income        3443 non-null float64
purpose             3443 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 349.7+ KB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,3443.000000,3443.000000,3443.000000,3443.000000,3443.000000,3443.000000,3443.000000
mean,0.128086,365003.491245,59.137090,0.914900,0.984606,0.052570,21940.394503
std,0.955201,21069.606065,7.563777,0.517016,1.316346,0.223207,12839.512522
min,-1.000000,328728.720605,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,346649.346146,56.000000,1.000000,0.000000,0.000000,13261.031000
50%,0.000000,365213.306266,60.000000,1.000000,0.000000,0.000000,18962.318000
75%,0.000000,383231.396871,64.000000,1.000000,2.000000,0.000000,27152.069000
max,20.000000,401755.400475,74.000000,4.000000,4.000000,1.000000,117616.523000


**Possible reasons for missing values in data**

There are not apparent and solid patterns for why data can be missing. Values are haphazardly amiss due to possibly two factors:

1) It makes sense that some retirees have missing values for income and days_employed. The data points aren't applicable for them anyways.
2) one's education level seems to have an influence on your income and days_employed: not all "secondary education" folks have a missing value in these columns, but all the missing columns store data of a person whose education is that of "secondary education". Maybe these null values indicated unemployment.

In [12]:
filtered = df.dropna()
filtered.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,19351.000000,19351.000000,19351.000000,19351.000000,19351.000000,19351.000000,19351.000000
mean,0.537388,63046.497661,43.255336,0.819079,0.972249,0.081184,26787.568355
std,1.371408,140827.311974,12.579170,0.550104,1.420596,0.273125,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [13]:
# Checking the distribution in the whole dataset
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


**Intermediate conclusion**

The distribution is not entirely the same, and there's some noticeable deviations with regard to days_employed, and total_income. This means that these missing values do impact our analysis and investigations, and we need to do something about them/drop them in order to arrive at the accurate conclusion.

**Intermediate conclusion**

I confirm that missing values are random, and thereby accidental. There's no strong correlation with other values in rows where they appear. Looking at line 168 and line 169, when I explored whether or not all retirees have missing income or employment value, the answer is no - there's no patterns. Furthermore, the missing values in the retiree group did nto significantly alter statistical distributions. The same applies to the missing values in total_income, which is checked below.

In [15]:
# Checking for other patterns - explain which: 
# checking to see if there are any patterns on missing values regarding EDUCATION LEVEL
education = df[df.education.str.startswith('se')]
education.info()
education.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13750 entries, 1 to 21524
Data columns (total 12 columns):
children            13750 non-null int64
days_employed       12342 non-null float64
dob_years           13750 non-null int64
education           13750 non-null object
education_id        13750 non-null int64
family_status       13750 non-null object
family_status_id    13750 non-null int64
gender              13750 non-null object
income_type         13750 non-null object
debt                13750 non-null int64
total_income        12342 non-null float64
purpose             13750 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 1.4+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,13750.000000,12342.000000,13750.000000,13750.0,13750.000000,13750.000000,12342.000000
mean,0.529309,72324.534813,44.502255,1.0,0.929091,0.089673,24616.530030
std,1.414925,148211.992345,12.419444,0.0,1.377369,0.285723,13735.872743
min,-1.000000,-18388.949901,0.000000,1.0,0.000000,0.000000,3306.762000
25%,0.000000,-2761.775723,35.000000,1.0,0.000000,0.000000,15638.180250
50%,0.000000,-1182.950507,44.000000,1.0,0.000000,0.000000,21862.935500
75%,1.000000,-228.599584,54.000000,1.0,1.000000,0.000000,30243.999500
max,20.000000,401755.400475,75.000000,1.0,4.000000,1.000000,276204.162000


In [16]:
education_dropped = education.dropna()
education_dropped.info()
education_dropped.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12342 entries, 1 to 21524
Data columns (total 12 columns):
children            12342 non-null int64
days_employed       12342 non-null float64
dob_years           12342 non-null int64
education           12342 non-null object
education_id        12342 non-null int64
family_status       12342 non-null object
family_status_id    12342 non-null int64
gender              12342 non-null object
income_type         12342 non-null object
debt                12342 non-null int64
total_income        12342 non-null float64
purpose             12342 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 1.2+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,12342.000000,12342.000000,12342.000000,12342.0,12342.000000,12342.000000,12342.000000
mean,0.528439,72324.534813,44.497407,1.0,0.931048,0.089532,24616.530030
std,1.406789,148211.992345,12.404062,0.0,1.379592,0.285521,13735.872743
min,-1.000000,-18388.949901,0.000000,1.0,0.000000,0.000000,3306.762000
25%,0.000000,-2761.775723,35.000000,1.0,0.000000,0.000000,15638.180250
50%,0.000000,-1182.950507,44.000000,1.0,0.000000,0.000000,21862.935500
75%,1.000000,-228.599584,54.000000,1.0,1.000000,0.000000,30243.999500
max,20.000000,401755.400475,75.000000,1.0,4.000000,1.000000,276204.162000


**Conclusions**

I confirm that missing values are random, and thereby accidental. There's no strong correlation with other values in rows where they appear. Looking at line 168 and line 169, when I explored whether or not all retirees have missing income or employment value, the answer is no - there's no patterns. Furthermore, the missing values in the retiree group did nto significantly alter statistical distributions. The same applies to the missing values in total_income, which is checked in line 171. 

The distribution is not entirely the same when we run it on the original dataset with missing values vs. the dataset without the missing values, and there's some noticeable deviations with regard to days_employed, and total_income. This means that these missing values do impact our analysis and investigations, and we need to drop them in order to arrive at the accurate conclusion. 

Since the days_employed and total_income are QUANTITATIVE missing values, we need to:
1. Determine whether your data has significant outliers.
2. If there aren't any significant outliers, сalculate the mean of your data — apply the mean() method to the column or the entire dataset.
3. If data has significant outliers, calculate the median of your data — apply the median() method to the column or the entire dataset.
4. Replace missing values with the mean or median using the fillna() method.

In addition to the 4 steps addressed above for missing values, I need to correct the spellings, look for implicit duplicates in the categories, consolidate reasons for taking out a loan, and figure out why some quantitative values are negative and whether or not that makes sense.

## Data transformation

Examine each column to see what issues we may have, beginning with removing duplicates and fixing educational information.

In [17]:
# Check to see all values in education column to check if and what spellings will need to be fixed

df['education'].value_counts()

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

In [18]:
# Fix the registers if required
df['education']=df['education'].str.lower()

In [19]:
# Checking the `education` column again to make sure it's fixed
df['education'].value_counts()

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

[Check the data the `children` column]

In [20]:
# Inspect the distribution of values in the `children` column
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

It is very strange that the minimum number of children in the family is negative, or has 20 children, or is a float number of children.
Since it is a quantitative variable, here's what I will do:
1. Calculate the mean of data — apply the mean() method to these values since there are not many of these outliers/issue data.
2. Round the median number to a sensible integer.
3. Replace children number is < 0, float, and 20 children with the rounded mean number (1). 

In [21]:
# [fix the data based on your decision]
mean_value = df['children'].mean()
print(mean_value)
mean_value2 = 1
df['children'].replace(20, 1, inplace =True)
df['children'].replace(-1, 1, inplace =True)
# df['children'] = np.where(df['children'].between(0.1,1), mean_value2, df['children'])

0.5389082462253194


In [22]:
# Checking the `children` column again to make sure it's fixed
df['children'].value_counts()

0    14149
1     4941
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

[Check the data in the `days_employed` column.]

In [23]:
# Find problematic data in `days_employed`, if they exist, and calculate the percentage
df['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

Negative/problematic values in days employed account for a significiant amount of data (15K+ rows). The most obvious and probable way this error could have happened, and the best corresponding way to address this error is by taking the absolute value of the days employed.

In [24]:
# Address the problematic values, if they exist
days_employed = df[df.days_employed < 0]
days_employed.count()
df['days_employed'] = df['days_employed'].abs()

In [25]:
# Check the result - make sure it's fixed
df['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

In [26]:
# Check the `dob_years` for suspicious values and count the percentage
df['dob_years'].describe()

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

Essentially, it is very strange that the minimum age of a customer who is taking out loans is 0 years old. Ideally, this number should be higher than 18 years old. Check for the amount of times this deviation occurs. If only a few times occur, replace 0 with mean. If this happens many times, it could be a missing value. Consider replacing with median.

In [27]:
# Address the issues in the `dob_years` column, if they exist
df[df.dob_years==0].count()
mean_value2 = df['dob_years'].mean()
df['dob_years'].replace(0, mean_value2, inplace=True)

In [28]:
# Check the result - make sure it's fixed
df['dob_years'].describe()

count    21525.000000
mean        43.496522
std         12.218174
min         19.000000
25%         34.000000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

[Check the `family_status` column for any data issues.]

In [29]:
# See the values for the column

df['family_status'].value_counts()
df['family_status'].describe()
df['family_status'].isnull().sum()

0

[Check the `gender` column for any data issues.]

In [32]:
# Check the values in the column
df['gender'].value_counts()
df[df.gender=='XNA'].count()
df['gender'].describe()

count     21525
unique        3
top           F
freq      14236
Name: gender, dtype: object

In [33]:
# Address the problematic values, if they exist
df[df.gender=='XNA']
df.drop(df.index[[10701,12]],inplace=True)

In [34]:
# Check the result - make sure it's fixed
df['gender'].value_counts()

F    14236
M     7287
Name: gender, dtype: int64

[Check the `income_type` column for data issues]

In [35]:
# Check the values in the column
df['income_type'].value_counts()

employee                       11119
business                        5084
retiree                         3855
civil servant                   1459
unemployed                         2
entrepreneur                       2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

In [36]:
# Address the problematic values, if they exist
## entrepreneur IS business, consolidate
df['income_type'].replace('entrepreneur', 'business', inplace =True)

In [37]:
# Check the result - make sure it's fixed
df['income_type'].value_counts()

employee                       11119
business                        5086
retiree                         3855
civil servant                   1459
unemployed                         2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

[Check to see if there are duplicates in our data.]

In [38]:
# Checking duplicates
df.duplicated().sum()

71

In [39]:
# Address the duplicates, if they exist
df=df.drop_duplicates()

In [40]:
# Last check whether we have any duplicates
df.duplicated().sum()

0

In [41]:
# Check the size of the dataset that you now have after your first manipulations with it
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21452 entries, 0 to 21524
Data columns (total 12 columns):
children            21452 non-null int64
days_employed       19350 non-null float64
dob_years           21452 non-null float64
education           21452 non-null object
education_id        21452 non-null int64
family_status       21452 non-null object
family_status_id    21452 non-null int64
gender              21452 non-null object
income_type         21452 non-null object
debt                21452 non-null int64
total_income        19350 non-null float64
purpose             21452 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 2.1+ MB



#### Summary:

1. Dropped 71 duplicates, now row numbers have reduced to 21452 rows.
2. Consolidated 'income type' by merging "entrepreneur" into "business" category.
3. Dropped 1 value of "XNA" in the gender column.
4. Replaced customer age value where it is "0" with the mean value of customer age.
5. Days employed column - took an absolute value of the entries to correct file errors.
6. Streamlined spelling for the Education column.
7. Replaced children values where it is negative with the mean value of children.


# Working with missing values

### Restoring missing values in `total_income`

Total_income and Days_employed have missing values that I need to address.
Strategies to fixing:

- Determine whether your data has significant outliers.
-  If there aren't any significant outliers, сalculate the mean of your data — apply the mean() method to the column or the entire dataset.
- If your data has significant outliers, calculate the median of your data — apply the median() method to the column or the entire dataset.
- Replace missing values with the mean or median using the fillna() method.

In [43]:
# Write a function that calculates the age category

def age_group(dob_years):
    if dob_years <= 18:
        return 'children'
    if 19 <= dob_years <= 30:
        return 'young adults'
    if 31 <= dob_years <= 50:
        return 'middle-aged adults'
    return 'senior adults'

In [44]:
# Test if the function works
print(age_group(40))
print(age_group(10))
print(age_group(60))
print(age_group(20))

middle-aged adults
children
senior adults
young adults


In [45]:
# Creating new column based on function
df['age_group'] = df['dob_years'].apply(age_group)

In [46]:
# Checking how values in the new column
print(df.head(10))

   children  days_employed  dob_years            education  education_id  \
0         1    8437.673028       42.0    bachelor's degree             0   
1         1    4024.803754       36.0  secondary education             1   
2         0    5623.422610       33.0  secondary education             1   
3         3    4124.747207       32.0  secondary education             1   
4         0  340266.072047       53.0  secondary education             1   
5         0     926.185831       27.0    bachelor's degree             0   
6         0    2879.202052       43.0    bachelor's degree             0   
7         0     152.779569       50.0  secondary education             1   
8         2    6929.865299       35.0    bachelor's degree             0   
9         0    2188.756445       41.0  secondary education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employee     0     40620.102   
1

Create a table that only has data without missing values. This data will be used to restore the missing values.

In [47]:
# Create a table without missing values and print a few of its rows to make sure it looks fine
df= df.dropna()
df.info()
print(df.head(20))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19350 entries, 0 to 21524
Data columns (total 13 columns):
children            19350 non-null int64
days_employed       19350 non-null float64
dob_years           19350 non-null float64
education           19350 non-null object
education_id        19350 non-null int64
family_status       19350 non-null object
family_status_id    19350 non-null int64
gender              19350 non-null object
income_type         19350 non-null object
debt                19350 non-null int64
total_income        19350 non-null float64
purpose             19350 non-null object
age_group           19350 non-null object
dtypes: float64(3), int64(4), object(6)
memory usage: 2.1+ MB
    children  days_employed  dob_years            education  education_id  \
0          1    8437.673028       42.0    bachelor's degree             0   
1          1    4024.803754       36.0  secondary education             1   
2          0    5623.422610       33.0  secondary edu

In [48]:
# Look at the mean values for income based on your identified factors
total_income_avg = df['total_income'].mean()
print(total_income_avg)
df['total_income'].describe()

26787.26668764858


count     19350.000000
mean      26787.266688
std       16475.822926
min        3306.762000
25%       16486.515250
50%       23201.873500
75%       32547.910750
max      362496.645000
Name: total_income, dtype: float64

In [49]:
# Look at the median values for income based on your identified factors
total_income_median = df['total_income'].median()
print(total_income_median)

23201.8735


I have chosen to replace the missing values with Median, because the dataset contains significant outliers for total_income.

In [50]:
#  Write a function that we will use for filling in missing values
df['total_income'].fillna(value=total_income_median,inplace=True)

def median_income (total_income):
    if total_income <= total_income_median:
        return 'Below Median Total Income'
    return 'Above Median Total Income'

In [51]:
# Check if it works
print(median_income(2))
print(median_income(200000))
df['total_income'].isnull().sum()

Below Median Total Income
Above Median Total Income


0

In [52]:
# Apply it to every row
df['median_income'] = df['total_income'].apply(median_income)

In [53]:
# Check if we got any errors
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,median_income
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,middle-aged adults,Above Median Total Income
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,middle-aged adults,Below Median Total Income
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,middle-aged adults,Above Median Total Income
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,middle-aged adults,Above Median Total Income
4,0,340266.072047,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,senior adults,Above Median Total Income
5,0,926.185831,27.0,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,young adults,Above Median Total Income
6,0,2879.202052,43.0,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,middle-aged adults,Above Median Total Income
7,0,152.779569,50.0,secondary education,1,married,0,M,employee,0,21731.829,education,middle-aged adults,Below Median Total Income
8,2,6929.865299,35.0,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,middle-aged adults,Below Median Total Income
9,0,2188.756445,41.0,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,middle-aged adults,Below Median Total Income


[Check that the total number of values in the `total_income`column matches the number of values in other ones post-application.]

In [55]:
# Checking the number of entries in the columns
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19350 entries, 0 to 21524
Data columns (total 14 columns):
children            19350 non-null int64
days_employed       19350 non-null float64
dob_years           19350 non-null float64
education           19350 non-null object
education_id        19350 non-null int64
family_status       19350 non-null object
family_status_id    19350 non-null int64
gender              19350 non-null object
income_type         19350 non-null object
debt                19350 non-null int64
total_income        19350 non-null float64
purpose             19350 non-null object
age_group           19350 non-null object
median_income       19350 non-null object
dtypes: float64(3), int64(4), object(7)
memory usage: 2.2+ MB


###  Restoring values in `days_employed`

In [56]:
# Distribution of `days_employed` medians based on your identified parameters
days_median=df['days_employed'].median()
print(days_median)

2194.218767670908


In [57]:
# Distribution of `days_employed` means based on your identified parameters
df['days_employed'].mean()

66918.06514084846

The large difference between mean and median show that my data has significant outliers, so to minimize impact to the dataset, one should apply the median() method to the column or the entire dataset.

In [58]:
# Write a function that calculates  medians based on identified parameter

def income_type (days_employed):
    if days_employed <= days_median:
        return 'Below Median Days Employed'
    return 'Above Median Days Employed'

In [59]:
# Check that the function works
print(income_type(20000))
print(income_type(2000))

Above Median Days Employed
Below Median Days Employed


In [60]:
# Apply function to the income_type
df['income_type'] = df['days_employed'].apply(income_type)

In [61]:
# Check if function worked
df.head(50)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,median_income
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,Above Median Days Employed,0,40620.102,purchase of the house,middle-aged adults,Above Median Total Income
1,1,4024.803754,36.0,secondary education,1,married,0,F,Above Median Days Employed,0,17932.802,car purchase,middle-aged adults,Below Median Total Income
2,0,5623.422610,33.0,secondary education,1,married,0,M,Above Median Days Employed,0,23341.752,purchase of the house,middle-aged adults,Above Median Total Income
3,3,4124.747207,32.0,secondary education,1,married,0,M,Above Median Days Employed,0,42820.568,supplementary education,middle-aged adults,Above Median Total Income
4,0,340266.072047,53.0,secondary education,1,civil partnership,1,F,Above Median Days Employed,0,25378.572,to have a wedding,senior adults,Above Median Total Income
5,0,926.185831,27.0,bachelor's degree,0,civil partnership,1,M,Below Median Days Employed,0,40922.170,purchase of the house,young adults,Above Median Total Income
6,0,2879.202052,43.0,bachelor's degree,0,married,0,F,Above Median Days Employed,0,38484.156,housing transactions,middle-aged adults,Above Median Total Income
7,0,152.779569,50.0,secondary education,1,married,0,M,Below Median Days Employed,0,21731.829,education,middle-aged adults,Below Median Total Income
8,2,6929.865299,35.0,bachelor's degree,0,civil partnership,1,F,Above Median Days Employed,0,15337.093,having a wedding,middle-aged adults,Below Median Total Income
9,0,2188.756445,41.0,secondary education,1,married,0,M,Below Median Days Employed,0,23108.150,purchase of the house for my family,middle-aged adults,Below Median Total Income


In [62]:
# Replacing missing values
df['days_employed'].fillna(value=days_median,inplace=True)

Checking to see that the total number of values in this column matches the number of values in other ones:

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19350 entries, 0 to 21524
Data columns (total 14 columns):
children            19350 non-null int64
days_employed       19350 non-null float64
dob_years           19350 non-null float64
education           19350 non-null object
education_id        19350 non-null int64
family_status       19350 non-null object
family_status_id    19350 non-null int64
gender              19350 non-null object
income_type         19350 non-null object
debt                19350 non-null int64
total_income        19350 non-null float64
purpose             19350 non-null object
age_group           19350 non-null object
median_income       19350 non-null object
dtypes: float64(3), int64(4), object(7)
memory usage: 2.2+ MB


## Categorization of data

Our project objective is to prepare a report for a bank’s loan division. I am asked to find out if a customer’s marital status and number of children have an impact on whether they will default on a loan. In addition, checking the customer's total income and their loan purpose would also be crucial in evaluating one's credit-worthiness. 

I chose these four values for further investigation. Family_status and children have been treated prior and already makes sense in terms of categorization. Therefore, I will focus on categorizing loan purpose and total income. 

In [64]:
# Print the values for your selected data for categorization
# children, family_status, total_income, purpose 
print(df[['children', 'family_status', 'purpose', 'total_income']])

       children      family_status                  purpose  total_income
0             1            married    purchase of the house     40620.102
1             1            married             car purchase     17932.802
2             0            married    purchase of the house     23341.752
3             3            married  supplementary education     42820.568
4             0  civil partnership        to have a wedding     25378.572
...         ...                ...                      ...           ...
21520         1  civil partnership     housing transactions     35966.698
21521         0            married        purchase of a car     24959.969
21522         1  civil partnership                 property     14347.610
21523         3            married        buying my own car     39054.888
21524         2            married             to buy a car     13127.587

[19350 rows x 4 columns]


In [65]:
df['family_status'].value_counts()

married              11143
civil partnership     3734
unmarried             2525
divorced              1083
widow / widower        865
Name: family_status, dtype: int64

In [66]:
df['purpose'].value_counts()

wedding ceremony                            721
to have a wedding                           693
having a wedding                            685
real estate transactions                    615
buy commercial real estate                  597
purchase of the house                       595
buying property for renting out             588
housing                                     587
transactions with commercial real estate    581
building a real estate                      580
housing transactions                        579
purchase of my own house                    574
property                                    572
purchase of the house for my family         570
building a property                         561
construction of own property                560
transactions with my real estate            559
buy real estate                             551
buy residential real estate                 546
housing renovation                          542
car                                     

In [67]:
df['total_income'].describe()

count     19350.000000
mean      26787.266688
std       16475.822926
min        3306.762000
25%       16486.515250
50%       23201.873500
75%       32547.910750
max      362496.645000
Name: total_income, dtype: float64

I would like to categorize the Purpose category into several broad ones:
1. Housing
2. Car
3. Education
4. Marriage
5. Other

In [68]:
# Write a function to categorize the data based on common topics
car = df.loc[df['purpose'].str.contains("car", case=False)]
marriage = df.loc[df['purpose'].str.contains("wedd|marr", case=False)]
house = df.loc[df['purpose'].str.contains("hous|estate", case=False)]
education = df.loc[df['purpose'].str.contains("educa", case=False)]

car_purpose=car['purpose'].count()
print(car_purpose)
marriage_purpose=marriage['purpose'].count()
print(marriage_purpose)
house_purpose=house['purpose'].count()
print(house_purpose)
education_purpose=education['purpose'].count()
print(education_purpose)
other_purpose = df['purpose'].count() - education_purpose - house_purpose - marriage_purpose - car_purpose
print(other_purpose)

3897
2099
7476
3157
2721


In the following code, I categorized numerical data:

In [70]:
# Looking through all the numerical data for categorization
print(df[['children', 'family_status', 'purpose', 'total_income']])

       children      family_status                  purpose  total_income
0             1            married    purchase of the house     40620.102
1             1            married             car purchase     17932.802
2             0            married    purchase of the house     23341.752
3             3            married  supplementary education     42820.568
4             0  civil partnership        to have a wedding     25378.572
...         ...                ...                      ...           ...
21520         1  civil partnership     housing transactions     35966.698
21521         0            married        purchase of a car     24959.969
21522         1  civil partnership                 property     14347.610
21523         3            married        buying my own car     39054.888
21524         2            married             to buy a car     13127.587

[19350 rows x 4 columns]


In [71]:
# Getting summary statistics for the column
df['total_income'].describe()

count     19350.000000
mean      26787.266688
std       16475.822926
min        3306.762000
25%       16486.515250
50%       23201.873500
75%       32547.910750
max      362496.645000
Name: total_income, dtype: float64

In the following segment, I decided on the ranges for grouping by total_income percentile.

In [72]:
# Creating function for categorizing into different numerical groups based on ranges
df['total_income'].describe()
def income_category (total_income):
    if total_income < 16486.515250:
        return 'Below 25% Percentile of Total Income'
    if 16486.515250 <= total_income < 23201.873500:
        return 'Below 50% Percentile of Total Income'
    if 23201.873500 <= total_income < 32547.910750:
        return 'Below 75% Percentile of Total Income'
    return 'Above 75% Percentile of Total Income'

In [73]:
# Creating column with categories
df['income_category'] = df['total_income'].apply(income_category)

In [74]:
# Count each categories values to see the distribution
df['income_category'].value_counts()

Above 75% Percentile of Total Income    4838
Below 25% Percentile of Total Income    4838
Below 50% Percentile of Total Income    4837
Below 75% Percentile of Total Income    4837
Name: income_category, dtype: int64

## Checking the Hypotheses


**Is there a correlation between having children and paying back on time?**

In [75]:
# Check the children data and paying back on time
x = df['children']
y = df['debt']
children_result = scipy.stats.linregress(x,y)
children_result.slope

Default = df[df['debt']==1].count()
print(Default)
# Calculating default-rate based on the number of children

children_0= df[(df['children']==0) & (df['debt']==1)].count()
print(children_0/Default*100)
children_1 = df[(df['children']==1) & (df['debt']==1)].count()
print(children_1/Default*100)
children_2 = df[(df['children']==2) & (df['debt']==1)].count()
print(children_2/Default*100)
children_3 = df[(df['children']==3) & (df['debt']==1)].count() 
print(children_3/Default*100)
children_4 = df[(df['children']==4) & (df['debt']==1)].count()
print(children_4/Default*100)
children_5 = df[(df['children']==5) & (df['debt']==1)].count()
print(children_5/Default*100)

# 0 child default case 952, 1 children default case 417, 2 children default 177, 3 children default 22, 
# 4 children default 3,  5 children 0, total default is 1571

children            1571
days_employed       1571
dob_years           1571
education           1571
education_id        1571
family_status       1571
family_status_id    1571
gender              1571
income_type         1571
debt                1571
total_income        1571
purpose             1571
age_group           1571
median_income       1571
income_category     1571
dtype: int64
children            60.598345
days_employed       60.598345
dob_years           60.598345
education           60.598345
education_id        60.598345
family_status       60.598345
family_status_id    60.598345
gender              60.598345
income_type         60.598345
debt                60.598345
total_income        60.598345
purpose             60.598345
age_group           60.598345
median_income       60.598345
income_category     60.598345
dtype: float64
children            26.543603
days_employed       26.543603
dob_years           26.543603
education           26.543603
education_id        26.5436

**Conclusion**

There is a correlation based on having children and paying back on time.
As the number of children increases, the default rate goes down.
total default is 1571.
0 child default case is 952, 1 children default case 417, 2 children default 177, 3 children default 22, 
4 children default 3,  and 5 children 0 cases.

** Default rate for the respective children number from 0 - 5 children is 61%, 27%, 11%, 1,4% and 0% **

**Is there a correlation between family status and paying back on time?**

In [76]:
# Check the family status data and paying back on time
print(df[['family_status', 'debt']])
df['family_status'].value_counts()

# Calculating default-rate based on family status
married = df[(df['family_status'] =='married') & (df['debt']==1)].count()
print(married/Default*100)
civil_partnership = df[(df['family_status'] =='civil partnership') & (df['debt']==1)].count()
print(civil_partnership/Default*100)
unmarried = df[(df['family_status'] == 'unmarried') & (df['debt']==1)].count()
print(unmarried/Default*100)
divorced = df[(df['family_status'] == 'divorced') & (df['debt']==1)].count() 
print(divorced/Default*100)
widow = df[(df['family_status'] == 'widow / widower') & (df['debt']==1)].count()
print(widow/Default*100)

           family_status  debt
0                married     0
1                married     0
2                married     0
3                married     0
4      civil partnership     0
...                  ...   ...
21520  civil partnership     0
21521            married     0
21522  civil partnership     1
21523            married     1
21524            married     0

[19350 rows x 2 columns]
children            53.85105
days_employed       53.85105
dob_years           53.85105
education           53.85105
education_id        53.85105
family_status       53.85105
family_status_id    53.85105
gender              53.85105
income_type         53.85105
debt                53.85105
total_income        53.85105
purpose             53.85105
age_group           53.85105
median_income       53.85105
income_category     53.85105
dtype: float64
children            21.578612
days_employed       21.578612
dob_years           21.578612
education           21.578612
education_id        21.578612
fa

**Conclusion**

There is correlation between family status and paying back on time.
Married has the highest default rate -- surprisingly, in the 50% range.
Civil Partnership comes in second -- roughly in the 20%
Divorced and Widow do not default as much, at below 5%.

**Is there a correlation between income level and paying back on time?**

In [77]:
# Check the income level data and paying back on time
df['income_category'].value_counts()

# Calculating default-rate based on income level
highest_quartile = df[(df['income_category'] =='Above 75% Percentile of Total Income') & (df['debt']==1)].count()
print(highest_quartile/Default*100)
second_quartile = df[(df['income_category'] =='Below 75% Percentile of Total Income') & (df['debt']==1)].count()
print(second_quartile/Default*100)
third_quartile = df[(df['income_category'] == 'Below 50% Percentile of Total Income') & (df['debt']==1)].count()
print(third_quartile/Default*100)
lowest_quartile = df[(df['income_category'] == 'Below 25% Percentile of Total Income') & (df['debt']==1)].count() 
print(lowest_quartile/Default*100)

children            21.70592
days_employed       21.70592
dob_years           21.70592
education           21.70592
education_id        21.70592
family_status       21.70592
family_status_id    21.70592
gender              21.70592
income_type         21.70592
debt                21.70592
total_income        21.70592
purpose             21.70592
age_group           21.70592
median_income       21.70592
income_category     21.70592
dtype: float64
children            27.116486
days_employed       27.116486
dob_years           27.116486
education           27.116486
education_id        27.116486
family_status       27.116486
family_status_id    27.116486
gender              27.116486
income_type         27.116486
debt                27.116486
total_income        27.116486
purpose             27.116486
age_group           27.116486
median_income       27.116486
income_category     27.116486
dtype: float64
children            26.798218
days_employed       26.798218
dob_years           26.79

**Conclusion**

There is not a strong correlation between income level and default rate - surprisingly!
All the default rates for representative quartiles even out around 25%.

**How does credit purpose affect the default rate?**

In [78]:
# Check the percentages for default rate for each credit purpose and analyze them
car_count = df.loc[df['purpose'].str.contains("car", case=False) & (df['debt']==1)].count()
marriage_count= df.loc[df['purpose'].str.contains("wedd|marr", case=False) & (df['debt']==1)].count()
house_count = df.loc[df['purpose'].str.contains("hous|estate",case=False) & (df['debt']==1)].count()
education_count = df.loc[df['purpose'].str.contains("educa", case=False) & (df['debt']==1)].count()

print(car_count/Default*100)
print(marriage_count/Default*100)
print(house_count/Default*100)
print(education_count/Default*100)

children            23.360917
days_employed       23.360917
dob_years           23.360917
education           23.360917
education_id        23.360917
family_status       23.360917
family_status_id    23.360917
gender              23.360917
income_type         23.360917
debt                23.360917
total_income        23.360917
purpose             23.360917
age_group           23.360917
median_income       23.360917
income_category     23.360917
dtype: float64
children            10.057288
days_employed       10.057288
dob_years           10.057288
education           10.057288
education_id        10.057288
family_status       10.057288
family_status_id    10.057288
gender              10.057288
income_type         10.057288
debt                10.057288
total_income        10.057288
purpose             10.057288
age_group           10.057288
median_income       10.057288
income_category     10.057288
dtype: float64
children            34.500318
days_employed       34.500318
dob_years 

**Conclusion**

Loans for Housing purposes had the biggest default rate at 35%, followed by Car loans.
It does seem that these two categories are statistically more prone to defaulting than education and wedding costs/loans. However, the difference is not too stark, and the gap is about 10-15%.

# General Conclusion 

[List your conclusions in this final section. Make sure you include all your important conclusions you made that led you to the way you processed and analyzed the data. Cover the missing values, duplicates, and possible reasons and solutions for problematic artifacts that you had to address.]

There is a correlation based on having children and paying back on time. As the number of children increases, the default rate goes down. total default is 1571. Default rate for the respective children number from 0 - 5 children is 61%, 27%, 11%, 1,4% and 0%.

There is correlation between family status and paying back on time.
Married has the highest default rate -- surprisingly, in the 50% range.
Civil Partnership comes in second -- roughly in the 20%
Divorced and Widow do not default as much, at below 5%.

There is NOT much correlation between income level and default rate - surprisingly!
All the default rates for representative quartiles even out around 25%.

Finally, loans for Housing purposes had the biggest default rate at 35%, followed by Car loans.
It does seem that these two categories are statistically more prone to defaulting than education and wedding costs/loans. However, the difference is not too stark, and the gap is about 10-15%.
